<a href="https://colab.research.google.com/github/abecode/learning-tensorflow/blob/master/tf_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""Roughly based on https://towardsdatascience.com/a-line-by-line-laymans-guide-to-linear-regression-using-tensorflow-3c0392aa9e1f

But using a single column vector matrix for the parameters instead of two variables.  

Also, using a matrix as input with a dummy ones column
"""

'Roughly based on https://towardsdatascience.com/a-line-by-line-laymans-guide-to-linear-regression-using-tensorflow-3c0392aa9e1f\n\nBut using a single column vector matrix for the parameters instead of two variables.  \n\nAlso, using a matrix as input with a dummy ones column\n'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
# 2021-04-23: I believe the previous stopped working when colab defaulted to a 
# different tf version
# import tensorflow as tf 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import matplotlib.pyplot as plt

In [4]:
x = np.random.randn(100) * 5 + 3 # *5 changes the sd, +3 changes the mean
err = np.random.randn(100) * .2 # *.2 changes tthe sd
y = 43*x + 10 + err # calculate observed response
y.shape = (100,1) # coerse to column vector
B= np.array([10, 43]) # represent population parameters as column vector
B.shape = (2,1) # coerse to column vector
X= np.column_stack((np.ones(100),x))
(X@B).shape # make sure the shapes are right


(100, 1)

In [5]:
tf.reset_default_graph() 
#x_ = tf.placeholder(tf.float32, shape=(None,), name="x")
#y_ = tf.placeholder(tf.float32, shape=(None,), name="y")
x_ = tf.placeholder(tf.float32, shape=(100,1), name="x")
y_ = tf.placeholder(tf.float32, shape=(100,1), name="y")
X_ = tf.placeholder(tf.float32, shape=X.shape, name="X")
# err_ = tf.placeholder("err", initializer=err)
#b = tf.get_variable("b", [1, 2])
b_ = tf.get_variable("b", shape=B.shape, initializer=tf.random_normal_initializer)
#b_ = tf.get_variable("b", shape=B.shape, initializer=tf.contrib.layers.xavier_initializer(uniform=True))
#b_ = tf.get_variable("b", shape=B.shape, initializer=tf.contrib.layers.xavier_initializer())
yhat_ = tf.matmul(X_, b_)
loss_ = tf.reduce_mean(tf.square(yhat_ - y_))
optimizer = tf.train.GradientDescentOptimizer(.025)
train_op = optimizer.minimize(loss_)

In [6]:
y_.shape

TensorShape([Dimension(100), Dimension(1)])

In [7]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    feed_dict = {X_: X , y_: y}
		
    for i in range(300):
      session.run(train_op, feed_dict)
      if i % 10 == 0:
        print(i, "loss:", loss_.eval(feed_dict))
    
    v = session.run(b_)

v

0 loss: 34698.277
10 loss: 50.940163
20 loss: 7.415131
30 loss: 3.2428188
40 loss: 1.4347049
50 loss: 0.6448338
60 loss: 0.29976937
70 loss: 0.1490259
80 loss: 0.08317298
90 loss: 0.05440419
100 loss: 0.04183671
110 loss: 0.0363463
120 loss: 0.033947807
130 loss: 0.032899536
140 loss: 0.032442234
150 loss: 0.032242674
160 loss: 0.03215498
170 loss: 0.03211721
180 loss: 0.03210057
190 loss: 0.032092784
200 loss: 0.03208962
210 loss: 0.03208848
220 loss: 0.03208766
230 loss: 0.032087464
240 loss: 0.03208717
250 loss: 0.032087434
260 loss: 0.03208715
270 loss: 0.03208739
280 loss: 0.032086827
290 loss: 0.032087326


array([[10.028934],
       [42.997383]], dtype=float32)

In [8]:
slope, intercept, r_value, p_value, std_err = stats.linregress(x,np.reshape(y, (100,)))
print(slope, intercept)
y_pred = x * slope + intercept
ms_err = np.mean(np.square(y.reshape((100,)) - y_pred))
print(ms_err)

42.99737958457639 10.028964995121157
0.032087320341771024


In [9]:
b_lsq, err, rank, sv = np.linalg.lstsq(X,y)
print(b_lsq, err, rank, sv)

[[10.028965  ]
 [42.99737958]] [3.20873203] 2 [58.43549668  9.00686288]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.
